# The Wedge

## Task 3: Building Summary Tables


## Python Modules

In [14]:
import os
import io
import shutil
import re
import datetime 
import csv

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

from zipfile import ZipFile # usually you'd do all these imports at the beginning

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

# Python program to illustrate the concept
# of threading
# importing the threading module
import threading
import time

from multiprocessing.pool import ThreadPool as Pool

import multiprocessing

import enum

import sqlite3


## Define Global Variables

In [22]:
# # Small File Sample
# zip_file_name = "WedgeZipOfZips_Small.zip"

## Full data Set
zip_file_name = "WedgeZipOfZips.zip"

# Clean data Set
# zip_file_name = "WedgeFiles_Clean.zip"

# Small Clean Data Set
# zip_file_name = "WedgeZipOfZips_Small_Clean.zip"

# CSV File Name
csv_file_name = "sample_transactions.csv"

# SQLite DB File Name
db_file_name = "summary_table.db"

# Working Directory included in .gitignore
# working_directory = "/media/psf/Home/Repos/BMKT670.V60-72020-Fall2022-Wedge-Project/eggs/"
working_directory = "/home/blackvwgolf95/BMKT670.V60-72020-Fall2022-Wedge-Project/eggs/"

conn = sqlite3.connect(working_directory + db_file_name)
print("Opened database successfully");

cursor = conn.cursor()

Opened database successfully


## Define Functions

In [3]:
def display_file_contents(file_name):
    # Ignore __MACOSX hidden files
    if(file_name.startswith( '__' )):
        return
    # Ignore folders
    if(file_name.endswith( '/' )):
        return

    print("File: ", file_name,
          " Size:", os.path.getsize(working_directory+file_name), 
          " ", convert_unit(os.path.getsize(working_directory+file_name), SIZE_UNIT.MB ) ), " MB"
    
#https://thispointer.com/python-get-file-size-in-kb-mb-or-gb-human-readable-format/
# Enum for size units
class SIZE_UNIT(enum.Enum):
   BYTES = 1
   KB = 2
   MB = 3
   GB = 4
def convert_unit(size_in_bytes, unit):
   """ Convert the size from bytes to other units like KB, MB or GB"""
   if unit == SIZE_UNIT.KB:
       return size_in_bytes/1024
   elif unit == SIZE_UNIT.MB:
       return size_in_bytes/(1024*1024)
   elif unit == SIZE_UNIT.GB:
       return size_in_bytes/(1024*1024*1024)
   else:
       return size_in_bytes

## GBQ Setup

In [4]:
# These first two values will be different on your machine. 
# service_path = "/Users/chandler/Dropbox/Teaching/"
# service_file = 'umt-msba-037daf11ee16.json' # change this to your authentication information  
# gbq_proj_id = 'umt-msba' # change this to your project. 
# service_path = "/media/psf/Home/Repos/"
service_path = "/home/blackvwgolf95/"
service_file = 'bmkt670-fall2022-wedge-project-6ce4398b80e4.json' # change this to your authentication information  
gbq_proj_id = 'bmkt670-fall2022-wedge-project' # change this to your project. 
dataset_id = 'wedgedataset'

# And this should stay the same. 
private_key = service_path + service_file

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

# for item in client.list_datasets() : 
#    print(item.full_dataset_id)

## Phase 3, Summary DB

1.	Sales by date by hour: By calendar date (YYYY-MM-DD) and hour of the day, determine the total spend in the store, the number of transactions, and a count of the number of items . 

In [34]:
query = """
    SELECT  EXTRACT( date FROM datetime) as date,
        EXTRACT( hour FROM datetime) as hour,
        ROUND( SUM(total), 2 ) as spend,        
        COUNT( DISTINCT
          CONCAT(
            CAST(EXTRACT(DATE FROM `datetime`) AS STRING), 
            CAST(`register_no` AS STRING ),
            CAST(`emp_no` AS STRING),
            CAST(`trans_no` AS STRING)
        ) ) AS `trans`,
        SUM( CASE WHEN `trans_status` IN ( 'V', 'R' ) THEN -1
             ELSE 1 END ) AS items 
    FROM `bmkt670-fall2022-wedge-project.wedgedataset.transArchive_*` 
    WHERE trans_status IN ( '', ' ', 'V', 'R', NULL ) AND `department` NOT IN (0, 15)
    GROUP BY date, hour
    ORDER BY DATE DESC, hour ASC
"""

transactions = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)

# transactions.head(12)

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39330/39330 [00:01<00:00, 20034.30rows/s]


In [36]:
# print(transactions.shape)
transactions.head(12)

,date,hour,spend,trans,items
0,2017-01-31,7,104.77,3,34
1,2017-01-31,8,1358.84,5,385
2,2017-01-31,9,2952.24,7,706
3,2017-01-31,10,4505.90,7,1106
4,2017-01-31,11,5246.76,8,1309
5,2017-01-31,12,6160.62,9,1428
6,2017-01-31,13,5389.13,10,1394
7,2017-01-31,14,5556.55,9,1416
8,2017-01-31,15,5843.35,10,1460
9,2017-01-31,16,5744.47,8,1443


In [33]:
transactions.to_sql('date_hour', con=conn, if_exists='replace')

39418

2.	Sales by owner by year by month: A file that has the following columns: card_no, year, month, sales, transactions, and items. 

In [41]:
query = """
    SELECT `card_no`, 
       EXTRACT( year FROM datetime ) AS `year`,
       EXTRACT( month FROM datetime ) AS `month`,
       SUM(total) AS `spend`,
       COUNT( DISTINCT
          CONCAT(
            CAST(EXTRACT(DATE FROM `datetime`) AS STRING), 
            CAST(`register_no` AS STRING ),
            CAST(`emp_no` AS STRING),
            CAST(`trans_no` AS STRING)
        ) ) AS `trans`,
      SUM( CASE WHEN `trans_status` IN ( 'V', 'R' ) THEN -1
             ELSE 1 END ) AS items
    FROM `bmkt670-fall2022-wedge-project.wedgedataset.transArchive_*`
    WHERE trans_status IN ( '', ' ', 'V', 'R', NULL ) AND `department` NOT IN (0, 15)
    GROUP BY card_no, year, month
    ORDER BY year, month, card_no
"""

transactions = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)

# transactions.head(12)

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 808896/808896 [00:37<00:00, 21456.39rows/s]


In [44]:
# print(transactions.shape)
transactions.head(12)

,card_no,year,month,spend,trans,items
0,10001,2010,1,252.92,5,96
1,10003,2010,1,91.76,1,22
2,10004,2010,1,62.77,1,5
3,10006,2010,1,147.71,8,67
4,10008,2010,1,138.34,3,42
5,10011,2010,1,377.51,11,121
6,10012,2010,1,18.97,1,3
7,10013,2010,1,24.52,1,9
8,10015,2010,1,30.65,1,7
9,10016,2010,1,99.94,2,43


In [45]:
transactions.to_sql('owner_spend_date', con=conn, if_exists='replace')

808896

3.	Sales by product description by year by month: A file that has the following columns: upc, description, department number, department name, year, month, sales, transactions, and items.

In [46]:
query = """
    SELECT `upc`,
       `description`,
       `bmkt670-fall2022-wedge-project.wedgedataset.transArchive_*`.`department`,
       `bmkt670-fall2022-wedge-project.wedgedataset.department_lookup`.`dept_name`,
       EXTRACT( year FROM datetime ) AS `year`,
       EXTRACT( month FROM datetime ) AS `month`,
       SUM(total) AS `sales`,
       COUNT( DISTINCT
          CONCAT(
            CAST(EXTRACT(DATE FROM `datetime`) AS STRING), 
            CAST(`register_no` AS STRING ),
            CAST(`emp_no` AS STRING),
            CAST(`trans_no` AS STRING)
        ) ) AS `trans`,
        SUM( CASE WHEN `trans_status` IN ( 'V', 'R' ) THEN -1
             ELSE 1 END ) AS `items`
    FROM `bmkt670-fall2022-wedge-project.wedgedataset.transArchive_*` 
    JOIN `bmkt670-fall2022-wedge-project.wedgedataset.department_lookup` ON ( `bmkt670-fall2022-wedge-project.wedgedataset.transArchive_*` .`department` = CAST( `bmkt670-fall2022-wedge-project.wedgedataset.department_lookup`.`department` AS INT ) )
    WHERE trans_status IN ( '', ' ')
    AND `trans_type` = "I"
    GROUP BY `upc`, `description`, `department`, `dept_name`, `year`, `month`
    ORDER BY `year` ASC, `month` ASC
"""

transactions = pandas_gbq.read_gbq(query,project_id = gbq_proj_id)

# transactions.head(12)

Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1090279/1090279 [01:16<00:00, 14160.88rows/s]


In [47]:
# print(transactions.shape)
transactions.head(4)

,upc,description,department,dept_name,year,month,sales,trans,items
0,0070587510017,O.Eve.Primrose Oil 60ct Barl,10,SUPPLEMENTS,2010,1,67.96,4,4
1,0005165109291,O.Creamy Pnut Butter 26oz Mar,1,PACKAGED GROCERY,2010,1,654.82,63,98
2,0009232533354,O.Oil&Vinegar Dressing 8oz An,1,PACKAGED GROCERY,2010,1,46.67,13,13
3,0087894100074,Orange CoQ10 4oz TrM,10,SUPPLEMENTS,2010,1,59.98,2,2


In [48]:
transactions.to_sql('product_date', con=conn, if_exists='replace')

1090279

## Write CSV

In [ ]:
transactions.to_csv(working_directory + csv_file_name, index=False)

In [ ]:
display_file_contents( csv_file_name )

# Cleanup ALL Local Files

In [21]:
# https://linuxize.com/post/python-delete-files-and-directories/
try:
    # shutil.rmtree(working_directory)
    conn.close()
    print('Done Cleanup')
    print("Completed Exit Code 0")
except OSError as e:
    print("Error: %s : %s" % (working_directory, e.strerror))
    print("Completed Exit Code -1")


Done Cleanup
Completed Exit Code 0
